<a href="https://colab.research.google.com/github/MinaKemmerLee/deep-learning-challenge/blob/main/deepLearningAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
columns_to_drop = ['EIN', 'NAME']
application_df = application_df.drop(columns=columns_to_drop)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
for column in application_df.columns:
    unique_count = application_df[column].nunique()
    print(f"{column}: {unique_count}")


APPLICATION_TYPE: 17
AFFILIATION: 6
CLASSIFICATION: 71
USE_CASE: 5
ORGANIZATION: 4
STATUS: 2
INCOME_AMT: 9
SPECIAL_CONSIDERATIONS: 2
ASK_AMT: 8747
IS_SUCCESSFUL: 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
value_counts = application_df['APPLICATION_TYPE'].value_counts().sort_index()

print("Desired output:")
print(value_counts)

Desired output:
T10      528
T12       27
T13       66
T14        3
T15        2
T17        1
T19     1065
T2        16
T25        3
T29        2
T3     27037
T4      1542
T5      1173
T6      1216
T7       725
T8       737
T9       156
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 500
application_types_to_replace = value_counts[value_counts < cutoff_value].index.tolist()

print("Application types to be replaced:")
print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

Application types to be replaced:
['T12', 'T13', 'T14', 'T15', 'T17', 'T2', 'T25', 'T29', 'T9']


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binningg
value_counts = application_df['CLASSIFICATION'].value_counts().sort_index()

print("Desired output:")
print(value_counts)

Desired output:
C0           3
C1000    17326
C1200     4837
C1230       36
C1234        2
         ...  
C7200       32
C7210        7
C8000       20
C8200       11
C8210        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
cutoff_value = 1

value_counts = application_df['CLASSIFICATION'].value_counts()

classes_to_replace = value_counts[value_counts < cutoff_value].index.tolist()


for cls in classes_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

class_value_counts = application_df['CLASSIFICATION'].value_counts()
print("Binned CLASSIFICATION value counts:")
print(class_value_counts)

Binned CLASSIFICATION value counts:
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 500
classifications_to_replace = value_counts[value_counts < cutoff_value].index.tolist()

print("Application types to be replaced:")
print(classifications_to_replace)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

Application types to be replaced:
['C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [18]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = [
    "APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE",
    "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS", "ASK_AMT"
]

dummy_df = pd.get_dummies(application_df, columns=categorical_columns)

dummy_df

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ASK_AMT_1665460552,ASK_AMT_1736232349,ASK_AMT_1893400128,ASK_AMT_2264109450,ASK_AMT_2310256039,ASK_AMT_3391919220,ASK_AMT_4653011914,ASK_AMT_5591584994,ASK_AMT_8556638692,ASK_AMT_8597806340
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34295,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34296,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34297,1,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = dummy_df.drop("IS_SUCCESSFUL", axis=1)
y = dummy_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(layers.Dense(64, activation='relu', input_shape=(8790,)))


# Second hidden layer
nn.add(layers.Dense(32, activation='relu'))


# Output layer
nn.add(layers.Dense(1, activation='sigmoid'))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                562624    
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 564,737
Trainable params: 564,737
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Train the model
nn.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
858/858 [==============================] - 14s 15ms/step - loss: 0.5726 - accuracy: 0.7206 - val_loss: 0.5643 - val_accuracy: 0.7220
Epoch 2/10
858/858 [==============================] - 11s 13ms/step - loss: 0.5188 - accuracy: 0.7587 - val_loss: 0.5761 - val_accuracy: 0.7061
Epoch 3/10
858/858 [==============================] - 10s 12ms/step - loss: 0.4318 - accuracy: 0.8042 - val_loss: 0.5968 - val_accuracy: 0.6870
Epoch 4/10
858/858 [==============================] - 11s 13ms/step - loss: 0.4051 - accuracy: 0.8096 - val_loss: 0.5878 - val_accuracy: 0.7115
Epoch 5/10
858/858 [==============================] - 12s 13ms/step - loss: 0.4006 - accuracy: 0.8097 - val_loss: 0.5941 - val_accuracy: 0.7168
Epoch 6/10
858/858 [==============================] - 11s 13ms/step - loss: 0.3979 - accuracy: 0.8115 - val_loss: 0.5973 - val_accuracy: 0.7165
Epoch 7/10
858/858 [==============================] - 12s 14ms/step - loss: 0.3967 - accuracy: 0.8107 - val_loss: 0.6079 - val_accuracy:

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 4.9646 - accuracy: 0.5433 - 1s/epoch - 5ms/step
Loss: 4.964637279510498, Accuracy: 0.5432944893836975


In [19]:
# Export our model to HDF5 file
model_filename = "my_model.h5"
nn.save(model_filename)
print(f"Model saved as {model_filename}")

Model saved as my_model.h5
